## Demonstration of DEM Download for 24k Quadrangle Maps ##

This notebook provides simple examples of how to search for, download, and merge geospatial data products within a 24k quadrangle from The National Map (TNM). Information about available quad names can be found at https://www.usgs.gov/faqs/where-can-i-find-indexes-usgs-topographic-maps.

In [ ]:
import sys
import os
sys.path.append('..') #path to the script we need (the dem_getter directory)

from dem_getter import dem_getter as dg #repository for functions to request/download/merge geospatial data from TNM

To fetch geospatial products, the user has to input the dataset type they are interested in. There are several datasets available:

In [ ]:
print('Available datasets are: {}'.format(list(dg.DATASETS_DICT.keys())))

More information about the available datasets can be found at https://www.usgs.gov/3d-elevation-program/about-3dep-products-services. A map showing product coverage is available at https://apps.nationalmap.gov/lidar-explorer/#/.

### Function Defaults and Optional Inputs ###

Each search is limited by a maximum number of products to be returned:

In [ ]:
print("The maximum number of products returned is: "+ str(dg.MAXITEMS))

Optionally, users can input a file name where the list of download paths can be saved, which the function will create if it doesn't already exist. By default, the code returns these paths as a list. 

In [ ]:
saveDlPaths=os.path.join('..','test_data','test_downloads')

The user can also specify the type of data they want returned. Different resolutions of data have the following options available:  

**STANDARD DEMS**  

* 1 meter DEM - GeoTIFF, IMG                 
    `Dataset code: 'DEM_1m'`
* 5 meter DEM (Alaska only) - Varies         
    `Dataset code: 'DEM_5m'`
* NED 1/9 arc-second (3 m) - IMG             
    `Dataset code: 'NED_1-9as'`
* NED 1/3 arc-second (10 m) - GeoTIFF        
    `Dataset code: 'NED_1-3as'`
* NED 1 arc-second (30 m) – GeoTIFF         
     `Dataset code: 'NED_1as'`
* NED 2 arc-second (Alaska – 60 m) - GeoTIFF   
     `Dataset code: 'NED_2as'`  

**SOURCE DATA PRODUCTS**  
  
* Lidar Point Cloud (LPC) – LAS, LAZ         
     `Dataset code: 'LPC'`  
* Original Product Resolution (OPR) - Varies  
     `Dataset code: 'OPR'`

Finally, some searches return products with the same spatial extent; the default setting for the parameter do_exclude_redundant_data returns only the most current version of the data.

###  Example Product Queries  ###

In [ ]:
#Data from the Andrews quad in Oregon
dg.get_aws_paths_from_24kQuadName(dataset='NED_1-3as', quadName='Andrews',stateName = 'Oregon',filePath= saveDlPaths,dataType="GeoTIFF")

Invalid quad names and searches that return no products will notify the user.

In [ ]:
#invalid quad name
try:
    dg.get_aws_paths_from_24kQuadName(dataset='NED_1-3as', quadName='NotaName', stateName='Notafornia')
except Exception as e:
    print(str(e))

In [ ]:
#combination of dataset and quad name that returns no results
try:
    dg.get_aws_paths_from_24kQuadName(dataset='DEM_5m', quadName='Andrews',stateName='Oregon')
except Exception as e:
    print(str(e))

Additionally, the function will halt if the user inputs an incorrect dataset or a datatype that doesn't go with a dataset.

In [ ]:
#searching for datasets not included in DATASETS_DICT raises a KeyError
#Here we are just 'catching' the error and printing as a string to avoid showing the full traceback
#in the notebook results
try:
    dl_list=dg.get_aws_paths_from_24kQuadName('bad_dataset','Andrews','Oregon')
except KeyError as e:
    print(str(e))

In [ ]:
#if searching for LPC products and datatype is specified, it must be LAS or LAZ, or LAS,LAZ.
#Quad map Kinikinik in Colorado

#Again, the try/except here is to avoid showing the full traceback
try:
    dg.get_aws_paths_from_24kQuadName('LPC','Kinikinik', 'Colorado',dataType='GeoTIFF')

except Exception as e:
    print(str(e))

In [ ]:
#same search as above with the correct data type input
dl_list=dg.get_aws_paths_from_24kQuadName('LPC','Kinikinik','Colorado', dataType='LAS,LAZ')
print("Preview of the first five products: "+str(dl_list[:5]))

We can also search for paths within multiple quads and compile a list of all the results.

In [ ]:
#test names to query-- adjacent quads in CO
quad_names = ['Kinikinik','Rustic', 'Big Narrows','Poudre Park']

dl_list=[]
for name in quad_names:
    paths= dg.get_aws_paths_from_24kQuadName(dataset='DEM_1m',quadName=name,stateName='Colorado')
    
    if paths: #skips if there are no available products
        for path in paths:
            dl_list.append(path) #master list of file paths

#adjacent quads might return some of the same data products;
# delete duplicates
dl_list_unique=[*set(dl_list)]
        
print("Number of products returned: "+str(len(dl_list)))
print("Number of unique products: "+str(len(dl_list_unique)))
print("Preview of the first five products: "+str(dl_list_unique[:5]))

### Downloading and Merging Data ###

The following section will demonstrate how to download and merge data, using a list of download paths that can be fetched with any of the functions described above.

To use the download function, the user must input:
* **List of download paths**
* **Folder name to save the data to**
    * If the input folder name does not exist, the function will create it
  
The batch_download function checks the size of the fetched data, and queries the user to continue with the download or not.

In [ ]:
#fetch paths
dl_list=dg.get_aws_paths_from_24kQuadName(dataset="NED_1-3as",
                                          quadName="Little Switzerland",
                                           stateName= 'North Carolina',
                                           filePath=None,
                                           dataType='GeoTIFF',
                                           doExcludeRedundantData=True)

#download
saved_paths=dg.batch_download(dl_list,os.path.join('..','test_data','test_downloads'))

These downloaded products can now be merged together. The merge function requires:
* **Input file list**
    * A list of all the filenames to merge
* **Output file path**
    * New file name and path to save the data as

In [ ]:
#You can also check the function doctstrings for more information
dg.merge_warp_dems?

In [ ]:
#now we'll merge together those downloaded files  
dg.merge_warp_dems(saved_paths,os.path.join('..','test_data','test_downloads','NCmerge.tif'))

The code below provides a quick way to fetch, download, and merge data from multiple quads.

In [ ]:
#list of two quad names
quad_names = ['Andrews', 'Juntura']
state = 'Oregon'
#download
full_filelist=[]
for name in quad_names:
    paths= dg.get_aws_paths_from_24kQuadName('NED_1-3as',name,'Oregon')

    if paths: #skips if polygon has no available products
        filelist=dg.batch_download(paths,os.path.join('..','test_data','test_downloads')) #downloads files
        for file in filelist:
            full_filelist.append(file) #master list of file paths for merging

dg.merge_warp_dems(full_filelist,os.path.join('..','test_data','test_downloads','quad_merge.tif'))